In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as stats
import statsmodels.stats.api as sms
from scipy.stats import norm

In [2]:
df = pd.read_csv('/kaggle/input/marketing-ab-testing/marketing_AB.csv')

In [3]:
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


In [4]:
df.shape

(588101, 7)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [6]:
df.columns

Index(['Unnamed: 0', 'user id', 'test group', 'converted', 'total ads',
       'most ads day', 'most ads hour'],
      dtype='object')

In [7]:
# Checking for balanced groups
group_counts = df['test group'].value_counts()
group_counts

test group
ad     564577
psa     23524
Name: count, dtype: int64

We have a big group imbalance, possible stratification.

In [8]:
# Determine sample size using power analysis
conversion_rate_ad = df[df['test group'] == 'ad']['converted'].mean()
conversion_rate_psa = df[df['test group'] == 'psa']['converted'].mean()

In [10]:
# Use the smaller of the two conversion rates for a conservative sample size estimate
conversion_rate_control = min(conversion_rate_ad, conversion_rate_psa)

In [12]:
minimum_detectable_effect = 0.02 # 2% increase in conversion rate
alpha = 0.05 # Significance level
power = 0.8 # Statisical power

In [14]:
# Caluclate Sample Size
effect_size = sms.proportion_effectsize(conversion_rate_control, conversion_rate_control + minimum_detectable_effect)
sample_size = sms.NormalIndPower().solve_power(effect_size, power=power, alpha=alpha, ratio=1)
sample_size = int(np.ceil(sample_size)) # round to nearest int

print(f"Required sample size per group: {sample_size}")

Required sample size per group: 1028


In [15]:
ad_group = df[df['test group'] == 'ad'].sample(n=sample_size, random_state=42)
psa_group = df[df['test group'] =='psa'].sample(n=sample_size, random_state=42)

In [17]:
ab_test_sample = pd.concat([ad_group, psa_group])

In [18]:
ab_test_sample['test group'].value_counts()

test group
ad     1028
psa    1028
Name: count, dtype: int64

In [24]:
ad_group.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
529666,529666,1300427,ad,False,21,Friday,20
385537,385537,1197483,ad,False,2,Thursday,20
120467,120467,1234257,ad,False,20,Sunday,10
186608,186608,1384841,ad,True,47,Friday,14
141292,141292,1646962,ad,False,13,Tuesday,13


In [25]:
# Calculate Conversion and Totals
ad_conversion = ad_group['converted'].sum()
psa_conversion = psa_group['converted'].sum()

ad_total = len(ad_group)
psa_total = len(psa_group)

In [26]:
# Perform the proportions z-test
z_stat, p_value = sms.proportions_ztest(
    [ad_conversion, psa_conversion],
    [ad_total, psa_total]
)

In [27]:
print(f" Z Statisitc: {z_stat}")
print(f"P Value: {p_value}")

 Z Statisitc: 0.8590379278678292
P Value: 0.39031959114413783


In [28]:
# Interpret Results
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. There is a statistical significant difference between the groups")
else:
    print("Fail to reject the null hypothesis. There is no statistically significant difference between the groups")

Fail to reject the null hypothesis. There is no statistically significant difference between the groups


In [29]:
# Calculate confidence intervals
def confidence_interval(successes, n, alpha=0.05):
    """Calculates confidence interval for a proportion."""
    proportion = successes / n
    z_critical = norm.ppf(1 - alpha / 2)
    margin_of_error = z_critical * (proportion * (1 - proportion) / n)**0.5
    lower_bound = proportion - margin_of_error
    upper_bound = proportion + margin_of_error
    return (lower_bound, upper_bound)

In [30]:
ad_ci = confidence_interval(ad_conversion, ad_total)
psa_ci = confidence_interval(psa_conversion, psa_total)

print(f"Ad Conversion Rate Confidence Interval: {ad_ci}")
print(f"Psa Conversion Rate Confidence Interval: {psa_ci}")

Ad Conversion Rate Confidence Interval: (0.017287025220663048, 0.03718768295054318)
Psa Conversion Rate Confidence Interval: (0.01255433386118113, 0.030247222559052333)


In [31]:
# Calculate conversion rates for print out.
ad_conversion_rate = ad_conversion / ad_total
psa_conversion_rate = psa_conversion / psa_total

print(f"Ad Conversion Rate: {ad_conversion_rate}")
print(f"Psa Conversion Rate: {psa_conversion_rate}")

Ad Conversion Rate: 0.027237354085603113
Psa Conversion Rate: 0.021400778210116732


In [1]:
# Reminders about the imbalance:
print("\nImportant Considerations:")
print("1. Remember the PSA group has a higher sampling rate. This can effect the overall test result.")
print("2. Consider using weighted analysis or other methods to account for the group imbalance.")


Important Considerations:
1. Remember the PSA group has a higher sampling rate. This can effect the overall test result.
2. Consider using weighted analysis or other methods to account for the group imbalance.


## Key Insights

1. **No Statistically Significant Difference**
   - We conducted an A/B test to evaluate the impact of the 'psa' strategy compared to the 'ad' strategy on conversion rates. Our statistical analysis, using a proportions z-test, revealed no statistically significant difference between the two groups (p-value = 0.39).
   - This means that, based on our sample data, we cannot confidently say that the 'psa' strategy leads to a different conversion rate than the 'ad' strategy.
2. **Conversion Rate Comparison**
  - While there was no statistically significant difference, the 'ad' group had a slightly higher conversion rate of 2.72% compared to the 'psa' group's 2.14%.
  - This difference, although not statistically significant, suggests that the 'ad' strategy performed slightly better in our sample.

3. **Confidence Intervals**
 - The 95% confidence interval for the 'ad' group's conversion rate is (1.73%, 3.72%), and for the 'psa' group, it is (1.26%, 3.02%).
 - The overlap between these intervals reinforces the lack of a statistically significant difference. It also shows the range of possible true conversion rates for each group.
 - Even at the higher end of the psa confidence interval, it does not exceed the higher end of the ad confidence interval.

4. **Group Imbalance and Limitations**
- It's crucial to acknowledge the significant imbalance between the 'ad' and 'psa' groups in our original dataset. The 'psa' group was considerably smaller, leading to a higher sampling rate in our test.
- This imbalance may have influenced our results and limits the generalizability of our findings. We recommend further analysis, potentially using weighted methods, to account for this imbalance.
- When applying these findings to the overall user base, the difference in group size must be taken into account

5. **Practical Significance and Next Steps**
- While there was a numerical difference in conversion rates, its practical significance should be carefully evaluated. We recommend a cost-benefit analysis to determine if the observed difference justifies any changes to our strategy.
- We suggest further investigation to explore potential reasons for the lack of a significant difference, such as further segmentation, or longer test duration.
- Given these results, we may choose to continue with the current 'ad' strategy, or to adjust the 'psa' strategy, or to conduct more tests.
- Further data collection might be needed to increase the statistical power of the test.